In [1]:
from transformers import pipeline
from transformers import PreTrainedTokenizerFast, AutoModelForSequenceClassification, TrainerCallback

2023-11-21 16:17:51.868714: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 16:17:52.442502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 16:17:53.897238: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from transformers import Trainer

In [3]:
import json
import os
import torch
from torch.nn import DataParallel
import wandb
import pandas as pd
from tokenizers import Tokenizer
from tokenizers.models import BPE
from datasets import load_dataset
from omegaconf import DictConfig
from typing import Any, Dict, List, Union

In [5]:

def tokenize_pad_and_truncate(texts: Dict[str, Any]) -> Dict[str, Any]:
    """Tokenizes, pads, and truncates input texts."""
    return wrapped_tokenizer(texts["slices"], truncation=True, padding="max_length", max_length=128)

tokenizer: Tokenizer = Tokenizer.from_file("/home/so87pot/n0w0f/structllm/src/structllm/tokenizer/tokenizer-slice_130k.json")
wrapped_tokenizer: PreTrainedTokenizerFast = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

test_data = load_dataset("csv", data_files="/home/so87pot/n0w0f/structllm/data/mb_1/csv/test_matbench_mp_e_form_0.csv")
tokenized_test_datasets = test_data.map(tokenize_pad_and_truncate, batched=True)

In [58]:
tokenized_test_datasets['train']

Dataset({
    features: ['slices', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 26544
})

In [7]:
tokenized_test_datasets['train'][0]

{'slices': 'Au N 0 1 - - - 0 1 - - o 0 1 o - - 0 1 - o - ',
 'labels': None,
 'input_ids': [89,
  29,
  7,
  8,
  6,
  6,
  6,
  7,
  8,
  6,
  6,
  53,
  7,
  8,
  53,
  6,
  6,
  7,
  8,
  6,
  53,
  6,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [59]:
trainer = Trainer(
            model=model.to("cuda"),
            data_collator=None,
            eval_dataset=tokenized_test_datasets
        )

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [60]:
prediction = trainer.predict(tokenized_test_datasets['train'])

In [64]:
type(prediction.predictions)

numpy.ndarray

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
            "/home/so87pot/n0w0f/structllm/src/structllm/models/finetune/checkpoints/train_matbench_mp_e_form_1/checkpoint-10000", 
            num_labels=1, 
            ignore_mismatched_sizes=True
        )

    
# Load the text classification pipeline with specified parameters
classifier = pipeline(
    task= "text-classification",
    model=model,
    tokenizer=wrapped_tokenizer,
    device=0 if torch.cuda.is_available() else -1,  # Set device to GPU if available
    framework="pt",  # Ensure PyTorch is used
)

In [8]:
classifier("Au N 0 1 - - - 0 1 - - o 0 1 o - - 0 1 - o -")

[{'label': 'LABEL_0', 'score': 0.8394883871078491}]